# Part 1: Nelder Mead

In [11]:
import numpy as np

np.random.seed(42) 
    
class Simplex:
    """ For generality """
    def __init__(self, vertices):        
        self.vertices = np.array(vertices)
        self.n_dim = self.vertices[0].shape[0]  # Number of dimensions
        
        assert self.vertices.shape[0] == self.n_dim + 1, "Simplex must have n+1 vertices in n dimensions"
        
    def __repr__(self):
        return f"Simplex(vertices={self.vertices})"
    
    def sort_simplex(self, func):
        vals = [func(vertex) for vertex in self.vertices]
        sorted_vertices = np.array([self.vertices[i] for i in np.argsort(vals)])
        self.vertices = sorted_vertices
            
    def centroid(self):
        centroid = np.mean(self.vertices[:-1], axis=0)  # Exclude the worst point      
        return centroid
    
    def converged(self, tol=1e-6):
        """ Check if the simplex has converged based on the tolerance. """
        return np.max(np.abs(self.vertices - np.mean(self.vertices, axis=0))) < tol

    def reflect(self, c, alpha):
        """Reflect worst point across the centriod - also used for extension and contraction"""
        x_w = self.vertices[-1]
        return c + alpha * (c - x_w)
    
    def contract(self, c, x_r, beta, mode="outside"):
        if mode == 'outside':
            return c + beta * (x_r - c)
        else:  # inside contraction
            return c - beta * (c - self.vertices[-1])
    
    def diameter(self):
        """Compute the maximum distance between any two vertices."""
        return np.max(np.linalg.norm(self.vertices - self.vertices[:, np.newaxis], axis=-1))
    
    def shrink(self, delta):
        self.vertices[1:] = self.vertices[0] + delta * (self.vertices[1:] - self.vertices[0])


 
    
alpha = 1.0  # Reflection
beta = 0.5 # Contraction
gamma = 2.0  # Expansion
delta = 0.5 # Shrinkage    
    
def nelder_mead(func, simplex: Simplex, max_iter=1000, tol=1e-6, verbose=False):
    """
    Perform the Nelder-Mead optimization algorithm in 2D.

    Parameters:
    - func: The objective function to minimize.
    - simplex: A list of points defining the initial simplex.
    - max_iter: Maximum number of iterations.
    - tol: Tolerance for convergence.

    Returns:
    - The point that minimizes the function.
    """

    for k in range(max_iter):
        simplex.sort_simplex(func)

        if simplex.diameter() < tol:
            print(f"Converged at iteration {k}.")
            break
        
        if verbose:
            best, worst = simplex.vertices[0], simplex.vertices[-1]
            print(f"iter {k:3d}, best={best}, f={func(best):.6f}, worst={worst}, f={func(worst):.6f}, diameter={simplex.diameter():.6f} ")

        centroid = simplex.centroid()
        
        # Reflection & Extension
        reflected = simplex.reflect(centroid, alpha)
        reflected_value = func(reflected)   # Dont recompute (can precompute for all vertices each iteration)
        
        if reflected_value < func(simplex.vertices[-2]):
            
            if reflected_value < func(simplex.vertices[0]): # Expand reflection if it was better than the second worst
                extended = simplex.reflect(centroid, gamma)  

                if func(extended) < reflected_value:
                    simplex.vertices[-1] = extended
                    
                else:
                    simplex.vertices[-1] = reflected
            else:
                simplex.vertices[-1] = reflected
            
            continue

        # Contraction
        if reflected_value >= func(simplex.vertices[-2]):
            # Outside contraction
            contracted = simplex.contract(centroid, reflected, beta, mode='outside')
            contracted_value = func(contracted)
            
            if contracted_value < reflected_value:
                simplex.vertices[-1] = contracted
            else:
                # Shrink
                simplex.shrink(delta)
            
        
        else:
            # Inside contraction
            contracted = simplex.contract(centroid, reflected, beta, mode='inside')
            contracted_value = func(contracted)
            
            if contracted_value < func(simplex.vertices[-1]):
                simplex.vertices[-1] = contracted
            else:
                # Shrink
                simplex.shrink(delta)
            
            
    return simplex.vertices[0]

### Testing GD functions

In [12]:
def f1(x):
    x, y, z = x
    return (x - z)**2 + (2*y + z)**2 + (4*x - 2*y + z)**2 + x + y

start_f1 = np.array([0,0,0], dtype=float)

In [13]:
def f2(x):
    x,y,z = x
    return (x - 1)**2 + (y - 1)**2 + 100*(y-x**2)**2 + 100*(z-y**2)**2

start_f2 = np.array([1.2, 1.2, 1.2], dtype=float)

In [14]:
def f3(x):
    x,y = x
    return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2

start_f3 = np.array([1,1], dtype=float)

In [ ]:
# x, y, (z) + f(x,y,(z)) # Approximate minimums of the functions
# f1 minimum: [-0.16232465 -0.22244489  0.16232465 -0.19775824]
# f2 minimum: [1.00400802 1.00400802 1.00400802 0.00327077]
# f3 minimum: [3.00000000e+00 5.00150015e-01 5.19139201e-07]

#### F1

In [16]:
simplex = Simplex([start_f1,
                   start_f1 + np.array([1, 0, 0], dtype=float),
                   start_f1 + np.array([0, 1, 0], dtype=float),
                   start_f1 + np.array([0, 0, 1], dtype=float)])

min_pt = nelder_mead(f1, simplex, max_iter=1000, tol=1e-7)
min_pt, f1(min_pt), simplex

Converged at iteration 152.


(array([-0.16666667, -0.22916667,  0.1666667 ]),
 np.float64(-0.19791666666666463),
 Simplex(vertices=[[-0.16666667 -0.22916667  0.1666667 ]
  [-0.16666664 -0.22916665  0.16666662]
  [-0.16666669 -0.22916669  0.16666666]
  [-0.16666665 -0.22916663  0.16666663]]))

#### F2

In [17]:
simplex = Simplex([start_f2,
                   start_f2 + np.array([1, 0, 0], dtype=float),
                   start_f2 + np.array([0, 1, 0], dtype=float),
                   start_f2 + np.array([0, 0, 1], dtype=float)])

min_pt = nelder_mead(f2, simplex, max_iter=1000, tol=1e-7)
min_pt, f2(min_pt)

Converged at iteration 201.


(array([0.99999999, 0.99999998, 0.99999997]),
 np.float64(3.5784604632461274e-16))

#### F3

In [18]:
simplex = Simplex([start_f3,
                   start_f3 + np.array([1, 0], dtype=float),
                   start_f3 + np.array([0, 1], dtype=float)])

min_pt = nelder_mead(f3, simplex, max_iter=1000, tol=1e-7)
min_pt, f3(min_pt)

Converged at iteration 94.


(array([2.99999995, 0.49999999]), np.float64(4.174865205948375e-16))

In [19]:
# Test for higher dimensions
def fx(x):
    x,y,z,a,b,c,d = x
    return (x - 1)**2 + (y - 1)**2 + (z - 1)**2 + (a - 1)**2 + (b - 1)**2 + (c - 1)**2 + (d - 1)**2

start_fx = np.array([1, 1, 1, 1, 1, 1, 1], dtype=float)
simplex = Simplex([start_fx,
                   start_fx + np.array([1, 0, 0, 0, 0, 0, 0], dtype=float),
                   start_fx + np.array([0, 1, 0, 0, 0, 0, 0], dtype=float),
                   start_fx + np.array([0, 0, 1, 0, 0, 0, 0], dtype=float),
                   start_fx + np.array([0, 0, 0, 1, 0, 0, 0], dtype=float),
                   start_fx + np.array([0, 0, 0, 0, 1, 0, 0], dtype=float),
                   start_fx + np.array([0, 0, 0, 0, 0, 1, 0], dtype=float),
                   start_fx + np.array([0, 0, 0, 0, 0, 0, 1], dtype=float)])
min_pt = nelder_mead(fx, simplex, max_iter=1000, tol=1e-7)
min_pt, fx(min_pt)

Converged at iteration 377.


(array([1., 1., 1., 1., 1., 1., 1.]), np.float64(0.0))

## Jeklo ruse

In [20]:
from itertools import combinations
import numpy as np

constraints = [
    (np.array([3, 1]), 600),   # C1: 3x1 + x2 = 600
    (np.array([2, 2]), 480),   # C2: 2x1 + 2x2 = 480
    (np.array([1, 0]), 0),     # C3: x1 = 0
    (np.array([0, 1]), 0)      # C4: x2 = 0
]

def is_feasible(x):
    x1, x2 = x
    return (
        3*x1 + x2 <= 600 + 1e-8 and     # C1
        2*x1 + 2*x2 <= 480 + 1e-8 and   # C2
        x1 >= -1e-8 and                 # Chack nonzero
        x2 >= -1e-8                     # Chack nonzero
    )

def objective(x1, x2):
    return 3*x1 + 2*x2

def jeklo_ruse():
    feasible_points = []

    # all combinations of 2 constraints
    for (A1, b1), (A2, b2) in combinations(constraints, 2):
        A = np.array([A1, A2])
        b = np.array([b1, b2])
        try:
            x = np.linalg.solve(A, b)
            if is_feasible(x):
                cost = objective(*x)
                feasible_points.append((x, cost))
        except np.linalg.LinAlgError:
            continue  # no intersection

    opt_point, opt_cost = max(feasible_points, key=lambda p: p[1])
    return opt_point, opt_cost 

In [21]:
jeklo_ruse()

(array([180.,  60.]), np.float64(660.0))

Q: What if there are m variables and n constrains? How many candidate points do we get? \
A: we can get n over m candidate points, each point is a solution to a system of m equality constraints, but not all are feasible (some will violate some of the n-m constraints)